In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
import geocoder
import sys
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [8]:
URL = "http://cocl.us/Geospatial_data"
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
#!pip install lxml  
soup = BeautifulSoup(r.content, 'html') 
table = soup.find('div', attrs = {'id':'container'}) 
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1
                
print('Data Collected.')
# define the dataframe columns

column_names = ['PostalCode', 'Borough', 'Neighborhood'] # The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)


# skip not assigned boroughs:
df = neighbors.drop(neighbors[(neighbors.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
neighbors.Neighborhood.replace("Not assigned", neighbors.Borough, inplace=True)

# copy Borough value to Neighborhood if NaN:
neighbors.Neighborhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
neighbors=neighbors.drop_duplicates()

print(neighbors.shape)
neighbors.head()




Data Collected.
(210, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [9]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [22]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_df.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [27]:
geo_merged = pd.merge(geo_df,neighbors, on='PostalCode')
geo_data=geo_merged[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
